In [1]:
%%js
$(".repository-content, .container").css("width", "90%");
$("body, #outer-dashboard, .navbar-default").css("background-color", "honeydew");
$(".navbar-default").css("background-color", "lightblue");

<IPython.core.display.Javascript object>

In [2]:
%%html
<img src="//localhost:3010/files/img/pydata.png">

In [3]:
%%js
Jupyter.notebook.kernel.comm_manager.register_target(
    'pydata_comm_channel', 
    function (comm, msg) {
        comm.send({url: window.location.href});
        console.log('Sending url');
    });

<IPython.core.display.Javascript object>

In [4]:
import datetime
from math import pi
import numpy as np

import zmq
from zmq.eventloop.zmqstream import ZMQStream

from ipywidgets import interact, widgets
from IPython.display import display
from ipykernel.comm import Comm
from urllib.parse import urlparse, parse_qs

from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource
from bokeh.models import DatetimeTickFormatter, HoverTool, DataRange1d
from bokeh.plotting import figure, show, output_notebook

In [5]:
output_notebook(hide_banner=True);

In [6]:
topic_widget = widgets.Text(description='Ticker:', placeholder='E.g. USDJPY=X')
display(topic_widget)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [7]:
def handler(*args):
    plot.set_topic(topic_widget.value)

topic_widget.on_submit(handler)

In [8]:
class LivePlot(object):
    PORT = 5556

    def __init__(self, topic=None):
        self.topic = topic or ''
        # See https://github.com/bokeh/bokeh/issues/6108
        self.source = ColumnDataSource(data={'x': [datetime.datetime.now()], 'y': [0]})
        self.plot = None

    def subscribe(self):
        ctx = zmq.Context()
        sock = ctx.socket(zmq.SUB)
        sock.connect('tcp://localhost:{}'.format(self.PORT))
        sock.setsockopt_string(zmq.SUBSCRIBE, '')
        stream = ZMQStream(sock)
        stream.on_recv(self.callback)

    def set_topic(self, topic):
        self.topic = topic.upper()
        self.source.data = dict(x=[], y=[])

    def make_plot(self):
        self.plot = figure(
            plot_height=400,
            plot_width=800,
            background_fill_alpha=0,
            border_fill_alpha=0,
            x_axis_type='datetime',
            logo=None,
            tools='pan,wheel_zoom,box_zoom,reset,resize,crosshair,hover'
        )
        self.plot.circle('x', 'y', source=self.source, color='steelblue', size=7)
        self.plot.line('x', 'y', source=self.source, color='crimson', alpha=0.5, line_width=2)
        self.plot.xaxis.major_label_orientation = pi / 4
        hover = self.plot.select(dict(type=HoverTool))
        hover.tooltips = [
            ("Value", "$y"),
        ]
        self.handle = show(self.plot, notebook_handle=True)

    def callback(self, msg):
        msg_text = msg[0].decode('utf-8').rpartition(' ')
        self.debug_msg = msg
        if self.topic == msg_text[0]:
            payload = float(msg_text[-1])
            x = datetime.datetime.now()
            self.source.data['x'].append(x)
            self.source.data['y'].append(payload)
            self.source.data['x'] = self.source.data['x'][-20:]
            self.source.data['y'] = self.source.data['y'][-20:]
            push_notebook(handle=self.handle)
            

In [9]:
plot = LivePlot()
plot.make_plot()
plot.subscribe()

In [10]:
def parse_url(url):
    q = urlparse(url).query
    return {k.lower(): v[0] for k, v in parse_qs(q).items()}

In [11]:
comm = Comm(target_name='pydata_comm_channel')

@comm.on_msg
def handle_msg(msg):
    url = msg['content']['data']['url']
    args = parse_url(url)
    ticker = args.get('ticker', '').upper()
    if ticker:
        topic_widget.value = ticker
        plot.set_topic(ticker)